<a href="https://colab.research.google.com/github/mahenoor22/skin-disease-build/blob/main/detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import random
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Input,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from glob import glob
from sklearn.model_selection import train_test_split

# **CallBack**

In [20]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>0.97):
            print("\nReached 97.0% accuracy so cancelling training!")
            self.model.stop_training = True

## **Import Dataset**

In [4]:
!git clone https://github.com/mahenoor22/skin-disease-build.git

fatal: destination path 'skin-disease-build' already exists and is not an empty directory.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
train_dir = r"/content/skin-disease-build/dataset/train"
test_dir=r"/content/skin-disease-build/dataset/test"
os.listdir(train_dir)

['Lupus and other Connective Tissue diseases',
 'Acne and Rosacea Photos',
 'Light Diseases and Disorders of Pigmentation',
 'Scabies Lyme Disease and other Infestations and Bites',
 'Warts Molluscum and other Viral Infections',
 'Cellulitis Impetigo and other Bacterial Infections',
 'Systemic Disease',
 'Bullous Disease Photos',
 'Melanoma Skin Cancer Nevi and Moles',
 'Herpes HPV and other STDs Photos',
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions',
 'Poison Ivy Photos and other Contact Dermatitis',
 'Urticaria Hives',
 'Psoriasis pictures Lichen Planus and related diseases',
 'Vascular Tumors',
 'Nail Fungus and other Nail Disease',
 'Hair Loss Photos Alopecia and other Hair Diseases',
 'Vasculitis Photos',
 'Seborrheic Keratoses and other Benign Tumors',
 'Atopic Dermatitis Photos',
 'Eczema Photos',
 'Tinea Ringworm Candidiasis and other Fungal Infections',
 'Exanthems and Drug Eruptions']

# **Image Generator**

In [8]:
data_generator = ImageDataGenerator(validation_split = 0.2)

In [10]:
train_gen = data_generator.flow_from_directory(train_dir,target_size=(150,150),
                                              batch_size=20,subset="training",
                                              class_mode='sparse')
valid_gen = data_generator.flow_from_directory(test_dir,target_size=(150,150),
                                              batch_size=20,subset="validation",
                                              class_mode='sparse')

Found 12453 images belonging to 23 classes.
Found 792 images belonging to 23 classes.


#**Model**

In [23]:
i = Input(shape = (150,150,3))
x = Conv2D(16,(3,3),activation='relu')(i)
x = MaxPooling2D((2,2))(x)
x = BatchNormalization()(x)
x = Conv2D(32,(3,3),activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64,(3,3),activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(64,(3,3),activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128,(3,3),activation='relu')(x)
x = BatchNormalization()(x)
x = Conv2D(128,(3,3),activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(64,activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(23,activation='softmax')(x)

model = Model(i,x)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 74, 74, 16)        64        
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
batch_normalization_13 (Batc (None, 72, 72, 32)        128       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 36, 36, 32)        0   

# **Compile and Run Model**

In [24]:
model.compile(optimizer = 'adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])

In [ ]:
history = model.fit_generator(train_gen,validation_data=valid_gen,epochs=3,callbacks=[myCallback()])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/3
469/623 [=====================>........] - ETA: 2:02 - loss: 3.2386 - accuracy: 0.1032